<a href="https://colab.research.google.com/github/largnor/Tensor_chatbot/blob/main/tensor_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import json
import pickle
import numpy as np
import os
import shutil

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay

lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json', encoding='utf-8').read())

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]

    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

initial_learning_rate = 0.01
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.96,
    staircase=True
)
sgd = SGD(learning_rate=lr_schedule, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

model_dir = 'chatbot_model'

if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
if os.path.exists(model_dir):
    shutil.rmtree(model_dir)

# Save the model
tf.saved_model.save(model, model_dir)

print("DONE")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Epoch 1/200
70/70 [==============================] - 1s 1ms/step - loss: 3.5465 - accuracy: 0.0401
Epoch 2/200
70/70 [==============================] - 0s 1ms/step - loss: 3.4675 - accuracy: 0.0716
Epoch 3/200
70/70 [==============================] - 0s 1ms/step - loss: 3.3962 - accuracy: 0.0946
Epoch 4/200
70/70 [==============================] - 0s 1ms/step - loss: 3.3331 - accuracy: 0.0802
Epoch 5/200
70/70 [==============================] - 0s 1ms/step - loss: 3.2028 - accuracy: 0.1146
Epoch 6/200
70/70 [==============================] - 0s 1ms/step - loss: 3.0527 - accuracy: 0.1662
Epoch 7/200
70/70 [==============================] - 0s 1ms/step - loss: 2.9139 - accuracy: 0.1977
Epoch 8/200
70/70 [==============================] - 0s 1ms/step - loss: 2.7070 - accuracy: 0.2636
Epoch 9/200
70/70 [==============================] - 0s 1ms/step - loss: 2.5533 - accuracy: 0.2693
Epoch 10/200
70/70 [==============================] - 0s 1ms/step - loss: 2.4076 - accuracy: 0.3181
Epoch 11/

DONE


In [10]:
import random
import json
import pickle
import numpy as np
import os
import shutil

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

import tensorflow as tf

lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

model_dir = 'chatbot_model'

model = tf.saved_model.load(model_dir)

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    bow_tensor = tf.convert_to_tensor(bow[np.newaxis, :], dtype=tf.float32)
    res = model(bow_tensor, training=False)
    res = tf.squeeze(res, axis=0)
    ERROR_THRESHOLD = 0.25
    result = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    result.sort(key = lambda x: x[1], reverse = True)
    return_list = []
    for r in result:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']

    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])  # 'responses' 키 사용
            break

    return result

print("BOT is RUNNING")

while True:
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


BOT is RUNNING
안녕
반가워
오늘 날씨가 어때?
비가 확 쏟아졌으면 좋겠어요
재미있는 이야기 없을까?
개가 사람을 가르친다’를 4자로 줄이면?.........  개인지도
고마워
네, 주문 넣었습니다.
고마워
금방 갖다 드릴게요.
고마워
금방 갖다 드릴게요.
감사합니다
저도 감사해요.


KeyboardInterrupt: ignored